In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
SQLALCHEMY_DATABASE_URL = "postgresql://robot-startml-ro:pheiph0hahj1Vaif@" \
                          "postgres.lab.karpov.courses:6432/startml"
engine = create_engine(SQLALCHEMY_DATABASE_URL)

In [3]:
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 500
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn,
                                       chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
        print(chunk_dataframe)
    conn.close()
    data = pd.concat(chunks, ignore_index=True)

    return data

In [4]:
def load_features() -> pd.DataFrame:
    query = "v_patrakeev_all_posts"
    query2 = "v_patrakeev_all_users"
    return batch_load_sql(query), batch_load_sql(query2)

In [5]:
posts, users = load_features()

     post_id  topic  text_length   likes  posts_views  likes_share
0       7319  movie          790   720.0       6585.0         10.0
1       7318  movie          728   680.0       6785.0         10.0
2       7317  movie          636   731.0       6803.0         10.0
3       7316  movie          800   677.0       6572.0         10.0
4       7315  movie          803  2619.0      16940.0         15.0
..       ...    ...          ...     ...          ...          ...
495     6801  movie         1162   742.0       6980.0         10.0
496     6800  movie          737   723.0       6697.0         10.0
497     6799  movie         1436   725.0       6734.0         10.0
498     6798  movie         1706   689.0       6681.0         10.0
499     6797  movie          916   691.0       6650.0         10.0

[500 rows x 6 columns]
     post_id  topic  text_length   likes  posts_views  likes_share
0       6796  movie          786   683.0       6717.0         10.0
1       6795  movie         1074   697

In [76]:
posts.head()

,post_id,topic,text_length,likes,posts_views,likes_share
0,7319,movie,790,720.0,6585.0,10.0
1,7318,movie,728,680.0,6785.0,10.0
2,7317,movie,636,731.0,6803.0,10.0
3,7316,movie,800,677.0,6572.0,10.0
4,7315,movie,803,2619.0,16940.0,15.0


In [7]:
users.head()

,user_id,age,city,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,168552,16,Ivanteyevka,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
1,168551,38,Moscow,0.138777,0.250000,0.400000,0.125000,0.000000,0.000000,0.000000,30.0,0.166667
2,168550,41,Yekaterinburg,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435
3,168549,18,Tula,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,34.0,0.029412
4,168548,36,Kaliningrad,0.138777,0.138045,0.141042,0.139542,0.131974,0.142722,0.116242,46.0,0.130435


Service operation scheme

In [103]:
#1 input id, time, limit, spisok_posts
id = 1000
spisok_posts = list(posts.post_id) #count all posts
limit = 10
time = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #time recome

In [104]:
#3 the list of posts user has seen
user_action_view = list(pd.read_sql(
     f"""
     select *
     from public.feed_data
     where user_id = {id} AND action = 'view'
         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )['post_id'])

In [105]:
#4 the list of all posts
all_posts = list(pd.read_sql(
     f"""
     select id
     from public.posts
         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )['id'])

In [106]:
#5 the list of posts user hasn't seen
spisok_posts = list(set(all_posts) - set(user_action_view))

In [107]:
#6 Create draft dataframe for preds
df = pd.DataFrame({'user_id': id,
                   'post_id': spisok_posts,
                   'time': time})

In [108]:
#7 add our features from download early
df = df.merge(posts,how='left',left_on='post_id', right_on='post_id')
df = df.merge(users,how='left',left_on='user_id', right_on='user_id')
df.head()

,user_id,post_id,time,topic,text_length,likes,posts_views,likes_share,age,city,business,covid,entertainment,movie,politics,sport,tech,views,ratio of likes posts
0,1000,1,2023-09-06 11:42:12,business,1967,1067.0,8508.0,12.0,46,Kayseri,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
1,1000,2,2023-09-06 11:42:12,business,2701,637.0,7495.0,8.0,46,Kayseri,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
2,1000,3,2023-09-06 11:42:12,business,3408,1122.0,8412.0,13.0,46,Kayseri,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
3,1000,4,2023-09-06 11:42:12,business,1026,1171.0,8158.0,14.0,46,Kayseri,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324
4,1000,5,2023-09-06 11:42:12,business,889,1153.0,8583.0,13.0,46,Kayseri,0.0,0.291667,0.0,0.344828,0.428571,0.444444,0.0,74.0,0.324324


In [96]:
#8 download our save model
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model = model.load_model('catboost_model', format="cbm")

In [109]:
#9 pred
pred = pd.DataFrame(model.predict_proba(df.drop(columns=['user_id','post_id', 'time']))[:, 1], columns=["predict"])

In [110]:
#add our
df = pd.concat([df, pred], axis=1)

In [111]:
df = df.sort_values(by=["predict"], ascending=False)

In [112]:
spisok = tuple(df["post_id"].head(10).tolist())
spisok

(1354, 1268, 1260, 1217, 1282, 1039, 1188, 1332, 1359, 1275)

In [113]:
post_top5_spisok = pd.read_sql(
    f"""
     select *
     from public.post
     where id IN {spisok}

         ;""",
     con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml"
 )


In [114]:
post_top5_spisok

,id,text,topic
0,1039,Tsunami debt deal to be announced\n\nChancello...,politics
1,1188,Minimum wage increased to £5.05\n\nThe minimum...,politics
2,1217,EU referendum could cost £80m\n\nIt could cost...,politics
3,1260,Short attacks US over tsunami aid\n\nFormer Ca...,politics
4,1268,MPs assess Scots fishing industry\n\nA group o...,politics
5,1275,Blair says UK tsunami aid to soar\n\nTony Blai...,politics
6,1282,Super union merger plan touted\n\nTwo of Brita...,politics
7,1332,Brown to outline presidency goals\n\nNext year...,politics
8,1354,Super union merger plan touted\n\nTwo of Brita...,politics
9,1359,Asylum children to face returns\n\nThe UK gove...,politics
